# 1. Mapa de Casos Encontrados

## Importações

In [1]:
import pandas as pd
import pydeck as pdk

### Lê os ``.csv``

DataSUS

In [2]:
cols = [
    'CO_MUN_RES','SG_UF', 'ID_MN_RESI'
]
    
datasus_df = pd.read_csv("../data/data_sus/INFLUD22-26-06-2025.csv", sep=";", usecols=cols)

len(datasus_df)

560577

In [3]:
data = datasus_df.rename(columns={
    "CO_MUN_RES": "Codigo_IBGE",
    "SG_UF": "Estado",
    "ID_MN_RESI": "Municipio"
})

data.dropna(subset=['Codigo_IBGE'], inplace=True)
data["Codigo_IBGE"] = data["Codigo_IBGE"].astype(int)

Estações

In [4]:
cols=[
    'Código IBGE do Município', 'Latitude', 'Longitude'
]

stations_df = pd.read_csv("../data/monitor_ar/EstacoesMonitorAr-Nov-2022.csv", sep=";", usecols=cols)


### Ajusta a variável ``coords``

In [5]:
coords = stations_df.rename(columns = {
    "Código IBGE do Município": "Codigo_IBGE"
})
coords['Codigo_IBGE'] = (coords['Codigo_IBGE'] / 10).astype(int)

coords['Latitude'] = coords['Latitude'].str.replace(',','.')
coords['Longitude'] = coords['Longitude'].str.replace(',','.')

# Converte lat/lon para numérico, tratando erros
coords['Latitude'] = pd.to_numeric(coords['Latitude'], errors='coerce')
coords['Longitude'] = pd.to_numeric(coords['Longitude'], errors='coerce')
coords.dropna(subset=['Latitude', 'Longitude', 'Codigo_IBGE'], inplace=True)

# Cria um DataFrame com UMA coordenada média por IBGE
coords = coords.groupby('Codigo_IBGE', as_index=False).agg(
    Latitude=('Latitude', 'mean'),
    Longitude=('Longitude', 'mean')
)
coords

,Codigo_IBGE,Latitude,Longitude
0,290570,-12.737628,-38.290757
1,291005,-12.636041,-38.340751
2,292950,-12.595482,-38.398561
3,310570,-20.275847,-43.073217
4,310620,-19.915781,-43.959763
...,...,...,...
67,430920,-29.923800,-50.987500
68,430930,-30.113800,-51.332000
69,431560,-32.069800,-52.094700
70,432200,-29.877300,-51.409100


## 2. Realizar o ``.merge`` (``JOIN``) entre os Dataframes

In [6]:
df = pd.merge(
    left=data,
    right=coords,
    on='Codigo_IBGE',
    how='inner'
)

df

,Estado,Municipio,Codigo_IBGE,Latitude,Longitude
0,SP,SAO CAETANO DO SUL,354880,-23.618443,-46.556354
1,SP,SAO CAETANO DO SUL,354880,-23.618443,-46.556354
2,SP,SAO PAULO,355030,-23.562650,-46.664135
3,SP,SAO CAETANO DO SUL,354880,-23.618443,-46.556354
4,SP,SAO PAULO,355030,-23.562650,-46.664135
...,...,...,...,...,...
158984,SP,TAUBATE,355410,-23.032351,-45.575805
158985,SP,TAUBATE,355410,-23.032351,-45.575805
158986,SP,CAMPINAS,350950,-22.907957,-47.078488
158987,SP,SANTOS,354850,-23.972176,-46.310840


## 3. Exibir o Mapa de Colunas

In [13]:
def render_map(df):
    df['coordinates'] = df[['Longitude', 'Latitude']].values.tolist()
    df = df[['Municipio', 'coordinates']]
    df = df.groupby('Municipio').agg(
        coordinates=('coordinates', 'first'),
        count=('Municipio', 'count')
    ).reset_index()

    layer = pdk.Layer(
        'ColumnLayer',
        data=df,
        get_position='coordinates',
        get_elevation='count',
        elevation_scale=10,
        radius=5000,
        get_fill_color=[255, 140, 0, 150],
        pickable=True,
        extruded=True,
    )

    view_state = pdk.ViewState(latitude=-23, longitude=-50, zoom=4, bearing=-45, pitch=45)

    r = pdk.Deck(
        layers=[layer],
        initial_view_state=view_state,
        tooltip={"text": "{Municipio} \nCount: {count}"},
    )
    display(r)

render_map(df)
    

{
  "initialViewState": {
    "bearing": -45,
    "latitude": -23,
    "longitude": -50,
    "pitch": 45,
    "zoom": 4
  },
  "layers": [
    {
      "@@type": "ColumnLayer",
      "data": [
        {
          "Municipio": "AMERICANA",
          "coordinates": [
            -47.347456,
            -22.724507
          ],
          "count": 710
        },
        {
          "Municipio": "ARACATUBA",
          "coordinates": [
            -50.439317,
            -21.186841
          ],
          "count": 554
        },
        {
          "Municipio": "ARARAQUARA",
          "coordinates": [
            -48.185832,
            -21.782522
          ],
          "count": 419
        },
        {
          "Municipio": "ARAUCARIA",
          "coordinates": [
            -49.386853,
            -25.561219
          ],
          "count": 927
        },
        {
          "Municipio": "BARRA LONGA",
          "coordinates": [
            -43.07321666666667,
            -20.275847
          ],
          "count": 12
        },
        {
          "Municipio": "BAURU",
          "coordinates": [
            -49.092759,
            -22.326608
          ],
          "count": 1477
        },
        {
          "Municipio": "BELO HORIZONTE",
          "coordinates": [
            -43.959763,
            -19.91578133333333
          ],
          "count": 15919
        },
        {
          "Municipio": "BETIM",
          "coordinates": [
            -44.15973233333333,
            -19.970310333333334
          ],
          "count": 2065
        },
        {
          "Municipio": "BRUMADINHO",
          "coordinates": [
            -44.155575,
            -20.142738666666666
          ],
          "count": 191
        },
        {
          "Municipio": "CAMACARI",
          "coordinates": [
            -38.29075675,
            -12.73762775
          ],
          "count": 466
        },
        {
          "Municipio": "CAMPINAS",
          "coordinates": [
            -47.07848833333333,
            -22.907957333333332
          ],
          "count": 9256
        },
        {
          "Municipio": "CANOAS",
          "coordinates": [
            -51.1448,
            -29.8833
          ],
          "count": 1324
        },
        {
          "Municipio": "CARAPICUIBA",
          "coordinates": [
            -46.83578,
            -23.531395
          ],
          "count": 663
        },
        {
          "Municipio": "CARIACICA",
          "coordinates": [
            -40.40218,
            -20.34201
          ],
          "count": 381
        },
        {
          "Municipio": "CATANDUVA",
          "coordinates": [
            -48.983075,
            -21.141943
          ],
          "count": 870
        },
        {
          "Municipio": "CONCEICAO DO MATO DENTRO",
          "coordinates": [
            -43.445101666666666,
            -18.943986
          ],
          "count": 25
        },
        {
          "Municipio": "CONGONHAS",
          "coordinates": [
            -43.84076183333334,
            -20.498477166666667
          ],
          "count": 113
        },
        {
          "Municipio": "CORONEL FABRICIANO",
          "coordinates": [
            -42.627828,
            -19.530986
          ],
          "count": 309
        },
        {
          "Municipio": "CUBATAO",
          "coordinates": [
            -46.39224266666667,
            -23.853344000000003
          ],
          "count": 234
        },
        {
          "Municipio": "CURITIBA",
          "coordinates": [
            -49.340368,
            -25.497588
          ],
          "count": 10349
        },
        {
          "Municipio": "DIADEMA",
          "coordinates": [
            -46.611622,
            -23.685876
          ],
          "count": 1555
        },
        {
          "Municipio": "DIAS D'AVILA",
          "coordinates": [
            -38.340751,
            -12.6360408
          ],
          "count": 130
        },
       